In [1]:
#!pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl numpy matplotlib spacy torchtext 

In [34]:
# Standard PyTorch imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy
from torch.autograd import Variable

# For plots
%matplotlib inline
import matplotlib.pyplot as plt


import tensorflow as tf

# !conda install torchtext spacy
# !python -m spacy download en
# !python -m spacy download de

from torchtext import data
from torchtext import datasets

import re
import spacy

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

url = re.compile('(<url>.*</url>)')


def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(url.sub('@URL@', text))]


def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]


# Testing IWSLT
DE = data.Field(tokenize=tokenize_de, init_token='<bos>', eos_token='<eos>', include_lengths=True)
EN = data.Field(tokenize=tokenize_en, init_token='<bos>', eos_token='<eos>', include_lengths=True)

train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN))


train_it = data.Iterator(train, batch_size=4, sort_within_batch=True, train=True, repeat=False, shuffle=True)
MIN_WORD_FREQ = 10
MAX_NUM_WORDS = 1000
DE.build_vocab(train.src, min_freq=MIN_WORD_FREQ)
EN.build_vocab(train.trg, max_size=MAX_NUM_WORDS)

DE.vocab.itos[:7]

In [39]:
tf.layers.dense

<function tensorflow.python.layers.core.dense>

In [40]:
tf.tanh

<function tensorflow.python.ops.math_ops.tanh>

In [42]:
X = tf.ones((5, 4, 3))

In [44]:
xflat = tf.reshape(X, (-1, 3))

In [45]:
xflat

<tf.Tensor 'Reshape:0' shape=(20, 3) dtype=float32>

In [52]:
xflat.shape[-1].value

3

In [56]:
bs, length, ndims = [v.value for v in X.shape]


In [57]:
bs

5

In [ ]:
class AttentionLayer(self, ndims_wd):
    def __init__(self, X):
        bs, length, ndims = [v.value for v in X.shape]
        xflat = tf.reshape(X, (-1, ndims))
        self.q, self.k, self.v = [tf.tanh(tf.layers.dense(xflat, ndims)) for _ in range(3)]
        self.s = self.k * self.q

class Encoder:
    def __init__(self, ndims_wd = 10, ndims_h = 20, n_layers = 2):
        pass
class Decoder:
    def __init__(self, ndims_wd = 10, ndims_h = 20, n_layers = 2):
        pass


class Transformer:
    def __init__(self, encoder, decoder):
        self.encoder = encoder
        self.decoder = decoder

In [38]:
for train_batch in train_it:
    src_tensor  = train_batch.src[0].data.cpu().numpy()
    src_len = train_batch.src[1].cpu().numpy()
    trg_tensor  = train_batch.trg[0].data.cpu().numpy()
    trg_len = train_batch.trg[1].cpu().numpy()
    print(src_tensor.shape, src_len.shape, trg_tensor.shape, trg_len.shape)
    print(src_tensor, src_len, trg_tensor, trg_len)
    break

(43, 4) (4,) (41, 4) (4,)
[[    2     2     2     2]
 [ 2076    39    47    42]
 [  390    70   262   145]
 [  280     9    10    43]
 [   11    18    60  1329]
 [    8   830    26     5]
 [  391     8 12128     3]
 [  647   306    94     1]
 [   95 27633     5     1]
 [ 2759  2270     3     1]
 [  588     0     1     1]
 [13254 35404     1     1]
 [    7     0     1     1]
 [38420 11307     1     1]
 [   90   306     1     1]
 [   25    21     1     1]
 [31794     0     1     1]
 [  537     5     1     1]
 [  419     3     1     1]
 [    4     1     1     1]
 [  367     1     1     1]
 [  830     1     1     1]
 [    4     1     1     1]
 [   10     1     1     1]
 [   17     1     1     1]
 [   21     1     1     1]
 [  433     1     1     1]
 [ 1015     1     1     1]
 [   33     1     1     1]
 [  262     1     1     1]
 [    4     1     1     1]
 [   26     1     1     1]
 [  135     1     1     1]
 [   12     1     1     1]
 [  649     1     1     1]
 [    4     1     1     1]
 [

In [27]:
train_batch


[torchtext.data.batch.Batch of size 10 from IWSLT]
	[.src]:('[torch.cuda.LongTensor of size 39x10 (GPU 0)]', '[torch.cuda.LongTensor of size 10 (GPU 0)]')
	[.trg]:('[torch.cuda.LongTensor of size 37x10 (GPU 0)]', '[torch.cuda.LongTensor of size 10 (GPU 0)]')

In [31]:
train_batch.src[0].data


     2      2      2      2      2      2      2      2      2      2
    24    100    547    142     66      0    143     39    998     39
    13      0  14119   7606    224      0     16      9   2188    812
   213     17     92      4     38     21     29    379     30     13
     4     21  12897    759     30      8    381     43   9236     22
    20   1039      4     28     31      0    152      0      4      5
    14     28     40     46   2044   1862    503  11915   4915      3
   186   5426     13      0     38     15     52   5067      5      1
    10     43   1139    221     15     11      4      0      3      1
   101      7      4     46      4     65     38    375      1      1
     4    279     20   1303     18      0     13    219      1      1
    49    234      6      7   1858     35     18     12      1      1
    14    349    393     25     11   3525   8801   3349      1      1
     6  28543     50   7393   6796    610    357      5      1      1
   110    360  1819

In [8]:
from torchtext.datasets import WMT14

In [9]:
WMT14('data/', ('.en', '.de'))

TypeError: __init__() missing 1 required positional argument: 'fields'

In [ ]:
# Load words from IWSLT

#!pip install torchtext spacy
#!python -m spacy download en
#!python -m spacy download de
from torchtext import data, datasets
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"
SRC = data.Field(tokenize=tokenize_de, pad_token=BLANK_WORD)
TGT = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, 
                 eos_token = EOS_WORD, pad_token=BLANK_WORD)

MAX_LEN = 100
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(SRC, TGT), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
MIN_FREQ = 1
SRC.build_vocab(train.src, min_freq=MIN_FREQ)
TGT.build_vocab(train.trg, min_freq=MIN_FREQ)

In [ ]:
dir(train)

In [ ]:
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl numpy matplotlib spacy torchtext seaborn 